In [1]:
from PyQt5.QtWidgets import QApplication, QLabel

app = QApplication([])
label = QLabel("PyQt5 Yüklü ve Çalışıyor!")
label.show()
app.exec_()

2024-12-08 22:39:31.200 Python[57583:2702256] +[IMKClient subclass]: chose IMKClient_Modern
2024-12-08 22:39:31.200 Python[57583:2702256] +[IMKInputSession subclass]: chose IMKInputSession_Modern


0

In [ ]:
import cv2
import mediapipe as mp
import sys
from PyQt5.QtWidgets import QApplication, QLabel, QVBoxLayout, QWidget
from PyQt5.QtGui import QImage, QPixmap
from PyQt5.QtCore import QTimer, Qt  # Qt buraya eklendi

# Mediapipe için modüller
mp_face_mesh = mp.solutions.face_mesh

# Kamera sınıfı
class CameraFeed(QWidget):
    def __init__(self):
        super().__init__()
        
        self.setWindowTitle("Test Arayüzü")
        self.setGeometry(100, 100, 800, 600)
        
        # Kamera görüntüsü için QLabel
        self.camera_label = QLabel(self)
        self.camera_label.resize(400, 300)
        
        # Arayüz alanı için QLabel
        self.interface_label = QLabel(self)
        self.interface_label.resize(400, 300)
        self.interface_label.move(400, 0)
        self.interface_label.setStyleSheet("background-color: lightgray; border: 1px solid black;")
        self.interface_label.setText("Arayüz Alanı")
        self.interface_label.setAlignment(Qt.AlignCenter)  # Qt burada kullanılıyor
        
        # Timer
        self.timer = QTimer(self)
        self.timer.timeout.connect(self.update_frame)
        
        # Mediapipe FaceMesh
        self.face_mesh = mp_face_mesh.FaceMesh(min_detection_confidence=0.7, min_tracking_confidence=0.7)
        
        # Kamera
        self.cap = cv2.VideoCapture(0)
        self.timer.start(20)  # 50 FPS
        
    def update_frame(self):
        ret, frame = self.cap.read()
        if not ret:
            return
        
        # BGR -> RGB
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # Mediapipe ile yüz analizi
        results = self.face_mesh.process(rgb_frame)
        
        if results.multi_face_landmarks:
            for face_landmarks in results.multi_face_landmarks:
                for id, landmark in enumerate(face_landmarks.landmark):
                    h, w, c = frame.shape
                    x, y = int(landmark.x * w), int(landmark.y * h)
                    
                    # Burun ucu koordinatları
                    if id == 1:  # Örneğin burun ucu
                        cv2.circle(frame, (x, y), 5, (0, 255, 0), -1)
        
        # Görüntüyü QLabel'e aktar
        h, w, ch = frame.shape
        bytes_per_line = ch * w
        qt_image = QImage(frame.data, w, h, bytes_per_line, QImage.Format_RGB888)
        pixmap = QPixmap.fromImage(qt_image)
        self.camera_label.setPixmap(pixmap)
        
    def closeEvent(self, event):
        self.cap.release()
        self.face_mesh.close()
        event.accept()


if __name__ == "__main__":
    app = QApplication(sys.argv)
    main_window = CameraFeed()
    main_window.show()
    sys.exit(app.exec_())

I0000 00:00:1733688137.429884 2731751 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 89.3), renderer: Apple M2
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1733688137.432908 2732059 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1733688137.435686 2732062 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
2024-12-08 23:02:18.940 Python[60162:2731751] +[IMKClient subclass]: chose IMKClient_Modern
2024-12-08 23:02:18.941 Python[60162:2731751] +[IMKInputSession subclass]: chose IMKInputSession_Modern
W0000 00:00:1733688139.047777 2732065 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.


KeyboardInterrupt: 

In [1]:
import cv2
import mediapipe as mp
import sys
from PyQt5.QtWidgets import QApplication, QLabel, QWidget
from PyQt5.QtGui import QImage, QPixmap
from PyQt5.QtCore import QTimer, Qt, QElapsedTimer

# Mediapipe için modüller
mp_face_mesh = mp.solutions.face_mesh

class CameraFeed(QWidget):
    def __init__(self):
        super().__init__()
        
        self.setWindowTitle("Test Arayüzü")
        self.setGeometry(100, 100, 1280, 720)
        
        # Kamera görüntüsü için QLabel
        self.camera_label = QLabel(self)
        self.camera_label.resize(640, 480)
        self.camera_label.setScaledContents(True)
        
        # Arayüz alanı için QLabel
        self.interface_label = QLabel(self)
        self.interface_label.resize(640, 480)
        self.interface_label.move(640, 0)
        self.interface_label.setStyleSheet("background-color: lightgray; border: 1px solid black;")
        self.interface_label.setText("Arayüz Alanı")
        self.interface_label.setAlignment(Qt.AlignCenter)
        
        # Timer
        self.timer = QTimer(self)
        self.timer.timeout.connect(self.update_frame)
        
        # Mediapipe FaceMesh
        self.face_mesh = mp_face_mesh.FaceMesh(min_detection_confidence=0.7, min_tracking_confidence=0.7)
        
        # Kamera
        self.cap = cv2.VideoCapture(0)
        self.cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
        self.cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
        self.timer.start(20)
        
        # İmleç pozisyonu ve seçim zamanı
        self.cursor_x, self.cursor_y = 0, 0
        self.start_time = None
        self.selected_area = None

    def update_frame(self):
        ret, frame = self.cap.read()
        if not ret:
            return

        # Görüntüyü yatay eksende çevir
        frame = cv2.flip(frame, 1)
        
        # BGR -> RGB
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # Mediapipe ile yüz analizi
        results = self.face_mesh.process(rgb_frame)
        
        if results.multi_face_landmarks:
            for face_landmarks in results.multi_face_landmarks:
                h, w, c = frame.shape
                nose = face_landmarks.landmark[1]
                self.cursor_x = int(nose.x * w)
                self.cursor_y = int(nose.y * h)
                
                # İmleç pozisyonu
                cv2.circle(frame, (self.cursor_x, self.cursor_y), 10, (255, 0, 0), -1)
                
                # Seçim alanı kontrolü
                self.check_selection()

        # Görüntüyü QLabel'e aktar
        h, w, ch = rgb_frame.shape
        bytes_per_line = ch * w
        qt_image = QImage(rgb_frame.data, w, h, bytes_per_line, QImage.Format_RGB888)
        pixmap = QPixmap.fromImage(qt_image)
        self.camera_label.setPixmap(pixmap)

    def check_selection(self):
        # İmleç arayüz alanında mı?
        if 640 <= self.cursor_x <= 1280 and 0 <= self.cursor_y <= 480:
            if self.start_time is None:
                self.start_time = QElapsedTimer()
                self.start_time.start()
            elif self.start_time.elapsed() > 3000:  # 3 saniye geçti mi?
                self.selected_area = "Arayüz Seçildi!"
                self.interface_label.setStyleSheet("background-color: lightblue; border: 1px solid black;")
                print(self.selected_area)
                self.start_time = None
        else:
            self.start_time = None
            self.interface_label.setStyleSheet("background-color: lightgray; border: 1px solid black;")

    def keyPressEvent(self, event):
        if event.key() == Qt.Key_Q:  # Kullanıcı 'q' tuşuna bastığında
            self.close()  # Pencereyi kapat
        
    def closeEvent(self, event):
        self.cap.release()
        self.face_mesh.close()
        event.accept()


if __name__ == "__main__":
    app = QApplication(sys.argv)
    main_window = CameraFeed()
    main_window.show()
    sys.exit(app.exec_())

I0000 00:00:1733688348.904055 2734787 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 89.3), renderer: Apple M2
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1733688348.906362 2734905 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1733688348.908846 2734904 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1733688350.462882 2734905 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.
2024-12-08 23:05:50.500 Python[60315:2734787] +[IMKClient subclass]: chose IMKClient_Modern
2024-12-08 23:05:50.500 Python[60315:2734787] +[IMKInputSession subclass]: chose IMKInputSession_Modern


SystemExit: 0

/Users/ulasyildiz/Documents/Gesture-Based Interaction/.venv/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [1]:
import cv2
import mediapipe as mp
import sys
from PyQt5.QtWidgets import QApplication, QLabel, QWidget, QVBoxLayout
from PyQt5.QtGui import QImage, QPixmap, QPainter, QColor
from PyQt5.QtCore import QTimer, Qt, QPoint

# Mediapipe için modüller
mp_face_mesh = mp.solutions.face_mesh

class CameraFeed(QWidget):
    def __init__(self):
        super().__init__()
        
        self.setWindowTitle("Göz İmleci ve Arayüz")
        self.setGeometry(100, 100, 1280, 720)
        
        # Ana layout
        self.layout = QVBoxLayout(self)
        
        # Kamera görüntüsü için QLabel
        self.camera_label = QLabel(self)
        self.camera_label.setFixedSize(1280, 480)
        self.camera_label.setStyleSheet("border: 1px solid black;")
        self.layout.addWidget(self.camera_label)
        
        # Arayüz alanı
        self.interface_label = QLabel(self)
        self.interface_label.setFixedSize(1280, 240)
        self.interface_label.setStyleSheet("background-color: lightgray; border: 1px solid black;")
        self.layout.addWidget(self.interface_label)
        
        # İmleç pozisyonu bilgisi
        self.cursor_label = QLabel(self.interface_label)
        self.cursor_label.setText("Göz İmleci Koordinatları: X: 0, Y: 0")
        self.cursor_label.setStyleSheet("font-size: 16px; color: green;")
        self.cursor_label.move(10, 10)

        # Timer
        self.timer = QTimer(self)
        self.timer.timeout.connect(self.update_frame)
        
        # Mediapipe FaceMesh
        self.face_mesh = mp_face_mesh.FaceMesh(min_detection_confidence=0.7, min_tracking_confidence=0.7)
        
        # Kamera
        self.cap = cv2.VideoCapture(0)
        self.cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
        self.cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
        self.timer.start(20)
        
        # Göz imleci pozisyonu
        self.cursor_x, self.cursor_y = 0, 0

    def update_frame(self):
        ret, frame = self.cap.read()
        if not ret:
            return

        # Görüntüyü yatay eksende çevir
        frame = cv2.flip(frame, 1)
        
        # BGR -> RGB
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # Mediapipe ile yüz analizi
        results = self.face_mesh.process(rgb_frame)
        
        if results.multi_face_landmarks:
            for face_landmarks in results.multi_face_landmarks:
                h, w, c = frame.shape
                nose = face_landmarks.landmark[1]
                self.cursor_x = int(nose.x * w)
                self.cursor_y = int(nose.y * h)
                
                # Göz imleci pozisyonu
                relative_x = self.cursor_x % self.interface_label.width()
                relative_y = self.cursor_y % self.interface_label.height()
                self.cursor_label.setText(f"Göz İmleci Koordinatları: X: {relative_x}, Y: {relative_y}")
                
                # İmleci arayüzde göster
                self.repaint_interface_label(relative_x, relative_y)

        # Görüntüyü QLabel'e aktar
        h, w, ch = rgb_frame.shape
        bytes_per_line = ch * w
        qt_image = QImage(rgb_frame.data, w, h, bytes_per_line, QImage.Format_RGB888)
        pixmap = QPixmap.fromImage(qt_image)
        self.camera_label.setPixmap(pixmap)
        
    def repaint_interface_label(self, x, y):
        pixmap = QPixmap(self.interface_label.size())
        pixmap.fill(Qt.transparent)
        
        painter = QPainter(pixmap)
        painter.setBrush(QColor(255, 0, 0))
        painter.setPen(Qt.NoPen)
        painter.drawEllipse(QPoint(x, y), 10, 10)
        painter.end()
        
        self.interface_label.setPixmap(pixmap)

    def keyPressEvent(self, event):
        if event.key() == Qt.Key_Q:  # Kullanıcı 'q' tuşuna bastığında
            self.close()  # Pencereyi kapat
        
    def closeEvent(self, event):
        self.cap.release()
        self.face_mesh.close()
        event.accept()


if __name__ == "__main__":
    app = QApplication(sys.argv)
    main_window = CameraFeed()
    main_window.show()
    sys.exit(app.exec_())

I0000 00:00:1733689852.074104 2745697 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 89.3), renderer: Apple M2
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1733689852.076313 2745790 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1733689852.078801 2745787 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
2024-12-08 23:30:53.584 Python[60858:2745697] +[IMKClient subclass]: chose IMKClient_Modern
2024-12-08 23:30:53.584 Python[60858:2745697] +[IMKInputSession subclass]: chose IMKInputSession_Modern
W0000 00:00:1733689853.616889 2745788 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.


SystemExit: 0

/Users/ulasyildiz/Documents/Gesture-Based Interaction/.venv/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [1]:
import cv2
import mediapipe as mp
import sys
from PyQt5.QtWidgets import QApplication, QLabel, QWidget, QVBoxLayout
from PyQt5.QtGui import QImage, QPixmap, QPainter, QColor
from PyQt5.QtCore import QTimer, Qt, QPoint

# Mediapipe için modüller
mp_face_mesh = mp.solutions.face_mesh

class CameraFeed(QWidget):
    def __init__(self):
        super().__init__()
        
        self.setWindowTitle("Göz İmleci ve Arayüz")
        self.setGeometry(100, 100, 1280, 720)
        
        # Ana layout
        self.layout = QVBoxLayout(self)
        
        # Kamera görüntüsü için QLabel
        self.camera_label = QLabel(self)
        self.camera_label.setFixedSize(320, 180)  # Daha küçük bir alan için
        self.camera_label.setStyleSheet("border: 1px solid black;")
        self.layout.addWidget(self.camera_label)
        
        # Arayüz alanı
        self.interface_label = QLabel(self)
        self.interface_label.setFixedSize(1280, 540)  # Arayüz için daha büyük alan
        self.interface_label.setStyleSheet("background-color: lightgray; border: 1px solid black;")
        self.layout.addWidget(self.interface_label)
        
        # İmleç pozisyonu bilgisi
        self.cursor_label = QLabel(self.interface_label)
        self.cursor_label.setText("Göz İmleci Koordinatları: X: 0, Y: 0")
        self.cursor_label.setStyleSheet("font-size: 16px; color: green;")
        self.cursor_label.move(10, 10)

        # Timer
        self.timer = QTimer(self)
        self.timer.timeout.connect(self.update_frame)
        
        # Mediapipe FaceMesh
        self.face_mesh = mp_face_mesh.FaceMesh(refine_landmarks=True,  # Daha hassas iris verileri
                                               min_detection_confidence=0.7, 
                                               min_tracking_confidence=0.7)
        
        # Kamera
        self.cap = cv2.VideoCapture(0)
        self.cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
        self.cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
        self.timer.start(20)
        
        # Göz imleci pozisyonu
        self.cursor_x, self.cursor_y = 0, 0

    def update_frame(self):
        ret, frame = self.cap.read()
        if not ret:
            return

        # Görüntüyü yatay eksende çevir
        frame = cv2.flip(frame, 1)
        
        # BGR -> RGB
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # Mediapipe ile yüz analizi
        results = self.face_mesh.process(rgb_frame)
        
        if results.multi_face_landmarks:
            for face_landmarks in results.multi_face_landmarks:
                h, w, c = frame.shape
                # Sağ ve sol iris merkezlerini alın
                left_eye_center = face_landmarks.landmark[468]  # Sol iris merkezi
                right_eye_center = face_landmarks.landmark[473]  # Sağ iris merkezi
                
                # Daha hassas hareket için faktör
                sensitivity_x = 1.8  # X ekseni hassasiyeti
                sensitivity_y = 1.8  # Y ekseni hassasiyeti
                self.cursor_x = int(left_eye_center.x * self.interface_label.width() * sensitivity_x)
                self.cursor_y = int(left_eye_center.y * self.interface_label.height() * sensitivity_y)
                
                # Koordinatları göster
                self.cursor_label.setText(f"Göz İmleci Koordinatları: X: {self.cursor_x}, Y: {self.cursor_y}")
                
                # İmleci arayüzde göster
                self.repaint_interface_label(self.cursor_x, self.cursor_y)

        # Görüntüyü QLabel'e aktar
        h, w, ch = rgb_frame.shape
        bytes_per_line = ch * w
        qt_image = QImage(rgb_frame.data, w, h, bytes_per_line, QImage.Format_RGB888)
        pixmap = QPixmap.fromImage(qt_image)
        self.camera_label.setPixmap(pixmap)
        
    def repaint_interface_label(self, x, y):
        pixmap = QPixmap(self.interface_label.size())
        pixmap.fill(Qt.transparent)
        
        painter = QPainter(pixmap)
        painter.setBrush(QColor(255, 0, 0))
        painter.setPen(Qt.NoPen)
        painter.drawEllipse(QPoint(x, y), 10, 10)  # İmleç boyutunu buradan ayarlayabilirsiniz
        painter.end()
        
        self.interface_label.setPixmap(pixmap)

    def keyPressEvent(self, event):
        if event.key() == Qt.Key_Q:  # Kullanıcı 'q' tuşuna bastığında
            self.close()  # Pencereyi kapat
        
    def closeEvent(self, event):
        self.cap.release()
        self.face_mesh.close()
        event.accept()


if __name__ == "__main__":
    app = QApplication(sys.argv)
    main_window = CameraFeed()
    main_window.show()
    sys.exit(app.exec_())

I0000 00:00:1733693530.409798 2795006 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 89.3), renderer: Apple M2
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1733693530.411788 2795090 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1733693530.416913 2795090 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1733693532.109072 2795087 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.
2024-12-09 00:32:12.146 Python[63356:2795006] +[IMKClient subclass]: chose IMKClient_Modern
2024-12-09 00:32:12.146 Python[63356:2795006] +[IMKInputSession subclass]: chose IMKInputSession_Modern


SystemExit: 0

/Users/ulasyildiz/Documents/Gesture-Based Interaction/.venv/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


Yedekte dursun aşağıdaki

In [1]:
import cv2
import mediapipe as mp
import numpy as np
import sys
from PyQt5.QtWidgets import QApplication, QLabel, QWidget
from PyQt5.QtGui import QImage, QPixmap, QPainter, QColor
from PyQt5.QtCore import QTimer, Qt, QPoint

mp_face_mesh = mp.solutions.face_mesh


class CameraFeed(QWidget):
    def __init__(self):
        super().__init__()

        self.setWindowTitle("Burun İmleci")
        self.setGeometry(100, 100, 1280, 720)

        # Kamera görüntüsü için QLabel
        self.camera_label = QLabel(self)
        self.camera_label.setFixedSize(320, 240)
        self.camera_label.setStyleSheet("border: 1px solid black;")
        self.camera_label.move((self.width() - self.camera_label.width()) // 2, 20)

        # Arayüz alanı
        self.interface_label = QLabel(self)
        self.interface_label.setFixedSize(1280, 480)
        self.interface_label.setStyleSheet("background-color: lightgray; border: 1px solid black;")
        self.interface_label.move(0, self.camera_label.height() + 40)

        # İmleç pozisyonu bilgisi
        self.cursor_label = QLabel(self)
        self.cursor_label.setText("Burun İmleci Koordinatları: X: 0, Y: 0")
        self.cursor_label.setStyleSheet("font-size: 16px; color: green;")
        self.cursor_label.setGeometry(10, self.interface_label.y() - 30, 400, 30)

        # Timer
        self.timer = QTimer(self)
        self.timer.timeout.connect(self.update_frame)

        # Mediapipe FaceMesh
        self.face_mesh = mp_face_mesh.FaceMesh(refine_landmarks=True, min_detection_confidence=0.7, min_tracking_confidence=0.7)

        # Kamera
        self.cap = cv2.VideoCapture(0)
        self.cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
        self.cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
        self.timer.start(20)

        # Burun imleci pozisyonu
        self.cursor_x, self.cursor_y = 0, 0

    def update_frame(self):
        ret, frame = self.cap.read()
        if not ret:
            return

        # Görüntüyü yatay eksende çevir
        frame = cv2.flip(frame, 1)

        # BGR -> RGB
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Mediapipe ile yüz analizi
        results = self.face_mesh.process(rgb_frame)

        if results.multi_face_landmarks:
            for face_landmarks in results.multi_face_landmarks:
                h, w, c = frame.shape
                nose_tip = face_landmarks.landmark[1]  # Burun ucu

                self.cursor_x = int(nose_tip.x * self.interface_label.width())
                self.cursor_y = int(nose_tip.y * self.interface_label.height())

                self.cursor_label.setText(f"Burun İmleci Koordinatları: X: {self.cursor_x}, Y: {self.cursor_y}")
                self.repaint_interface_label(self.cursor_x, self.cursor_y)

        h, w, ch = rgb_frame.shape
        bytes_per_line = ch * w
        qt_image = QImage(rgb_frame.data, w, h, bytes_per_line, QImage.Format_RGB888)
        pixmap = QPixmap.fromImage(qt_image)
        self.camera_label.setPixmap(pixmap)

    def repaint_interface_label(self, x, y):
        pixmap = QPixmap(self.interface_label.size())
        pixmap.fill(Qt.transparent)

        painter = QPainter(pixmap)
        painter.setBrush(QColor(255, 0, 0))
        painter.setPen(Qt.NoPen)
        painter.drawEllipse(QPoint(x, y), 20, 20)
        painter.end()

        self.interface_label.setPixmap(pixmap)

    def keyPressEvent(self, event):
        if event.key() == Qt.Key_Q:
            self.close()

    def closeEvent(self, event):
        self.cap.release()
        self.face_mesh.close()
        event.accept()


if __name__ == "__main__":
    app = QApplication([])
    main_window = CameraFeed()
    main_window.show()
    sys.exit(app.exec_())

I0000 00:00:1733693511.211304 2794535 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 89.3), renderer: Apple M2
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1733693511.212966 2794637 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1733693511.217650 2794637 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1733693512.900284 2794639 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.
2024-12-09 00:31:52.943 Python[63337:2794535] +[IMKClient subclass]: chose IMKClient_Modern
2024-12-09 00:31:52.943 Python[63337:2794535] +[IMKInputSession subclass]: chose IMKInputSession_Modern


SystemExit: 0

/Users/ulasyildiz/Documents/Gesture-Based Interaction/.venv/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


Kalibrasyon Edilmiş Hali Aşağıda

In [1]:
import cv2
import mediapipe as mp
import numpy as np
import sys
from PyQt5.QtWidgets import QApplication, QLabel, QWidget
from PyQt5.QtGui import QImage, QPixmap, QPainter, QColor
from PyQt5.QtCore import QTimer, Qt, QPoint

mp_face_mesh = mp.solutions.face_mesh


class CameraFeed(QWidget):
    def __init__(self):
        super().__init__()

        self.setWindowTitle("Burun İmleci")
        self.setGeometry(100, 100, 1280, 720)

        # Kamera görüntüsü için QLabel
        self.camera_label = QLabel(self)
        self.camera_label.setFixedSize(320, 240)
        self.camera_label.setStyleSheet("border: 1px solid black;")
        self.camera_label.move((self.width() - self.camera_label.width()) // 2, 20)

        # Arayüz alanı
        self.interface_label = QLabel(self)
        self.interface_label.setFixedSize(1280, 400)  # Yükseklik 400'e ayarlandı
        self.interface_label.setStyleSheet("background-color: lightgray; border: 1px solid black;")
        self.interface_label.move(0, self.camera_label.height() + 40)

        # İmleç pozisyonu bilgisi
        self.cursor_label = QLabel(self)
        self.cursor_label.setText("Burun İmleci Koordinatları: X: 640, Y: 200")  # Başlangıçta ortada
        self.cursor_label.setStyleSheet("font-size: 16px; color: green;")
        self.cursor_label.setGeometry(10, self.interface_label.y() - 30, 400, 30)

        # Timer
        self.timer = QTimer(self)
        self.timer.timeout.connect(self.update_frame)

        # Mediapipe FaceMesh
        self.face_mesh = mp_face_mesh.FaceMesh(refine_landmarks=True, min_detection_confidence=0.7, min_tracking_confidence=0.7)

        # Kamera
        self.cap = cv2.VideoCapture(0)
        self.cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
        self.cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
        self.timer.start(20)

        # Hassasiyet çarpanları
        self.x_sensitivity = 3
        self.y_sensitivity = 5

        # Burun imleci pozisyonu (Başlangıçta ortada)
        self.cursor_x = self.interface_label.width() // 2
        self.cursor_y = self.interface_label.height() // 2

        # Kamera yüzü ortalamak için kesme oranları
        self.crop_x = 0.25  # %25 sağdan ve soldan kırp
        self.crop_y = 0.15  # %15 yukarıdan ve aşağıdan kırp

    def update_frame(self):
        ret, frame = self.cap.read()
        if not ret:
            return

        # Görüntüyü yatay eksende çevir
        frame = cv2.flip(frame, 1)

        # Görüntüyü ortalamak için kırp
        h, w, c = frame.shape
        x_start = int(w * self.crop_x)
        x_end = w - x_start
        y_start = int(h * self.crop_y)
        y_end = h - y_start
        cropped_frame = frame[y_start:y_end, x_start:x_end]

        # BGR -> RGB
        rgb_frame = cv2.cvtColor(cropped_frame, cv2.COLOR_BGR2RGB)

        # Mediapipe ile yüz analizi
        results = self.face_mesh.process(rgb_frame)

        if results.multi_face_landmarks:
            for face_landmarks in results.multi_face_landmarks:
                h, w, c = cropped_frame.shape
                nose_tip = face_landmarks.landmark[1]  # Burun ucu

                # Kullanıcının burun koordinatlarını hesapla
                new_cursor_x = int(nose_tip.x * self.interface_label.width())
                new_cursor_y = int(nose_tip.y * 400)  # Y değeri 0-400 ile sınırlı

                # İmlecin başlangıç pozisyonunda hassasiyet uygulanmasın
                if new_cursor_x != self.cursor_x or new_cursor_y != self.cursor_y:
                    self.cursor_x = int((new_cursor_x - self.interface_label.width() // 2) * self.x_sensitivity + self.interface_label.width() // 2)
                    self.cursor_y = int((new_cursor_y - self.interface_label.height() // 2) * self.y_sensitivity + self.interface_label.height() // 2)

                # İmleci sınırlar içinde tut
                self.cursor_x = max(0, min(self.cursor_x, self.interface_label.width()))
                self.cursor_y = max(0, min(self.cursor_y, 400))  # Y sınırı 0-400 arasında

                # İmleç pozisyonu bilgisini güncelle
                self.cursor_label.setText(f"Burun İmleci Koordinatları: X: {self.cursor_x}, Y: {self.cursor_y}")
                self.repaint_interface_label(self.cursor_x, self.cursor_y)

        # Kamera görüntüsünü PyQt5 QLabel'e aktar
        h, w, ch = rgb_frame.shape
        bytes_per_line = ch * w
        qt_image = QImage(rgb_frame.data, w, h, bytes_per_line, QImage.Format_RGB888)
        pixmap = QPixmap.fromImage(qt_image)
        self.camera_label.setPixmap(pixmap)

    def repaint_interface_label(self, x, y):
        pixmap = QPixmap(self.interface_label.size())
        pixmap.fill(Qt.transparent)

        painter = QPainter(pixmap)
        painter.setBrush(QColor(255, 0, 0))
        painter.setPen(Qt.NoPen)
        painter.drawEllipse(QPoint(x, y), 20, 20)
        painter.end()

        self.interface_label.setPixmap(pixmap)

    def keyPressEvent(self, event):
        if event.key() == Qt.Key_Q:
            self.close()

    def closeEvent(self, event):
        self.cap.release()
        self.face_mesh.close()
        event.accept()


if __name__ == "__main__":
    app = QApplication([])
    main_window = CameraFeed()
    main_window.show()
    sys.exit(app.exec_())

I0000 00:00:1733696596.838574 2834685 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 89.3), renderer: Apple M2
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1733696596.840165 2834880 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1733696596.844881 2834878 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
2024-12-09 01:23:18.499 Python[65023:2834685] +[IMKClient subclass]: chose IMKClient_Modern
2024-12-09 01:23:18.499 Python[65023:2834685] +[IMKInputSession subclass]: chose IMKInputSession_Modern
W0000 00:00:1733696598.536335 2834883 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.


SystemExit: 0

/Users/ulasyildiz/Documents/Gesture-Based Interaction/.venv/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


Arayüze tuş ekleme

In [1]:
import cv2
import mediapipe as mp
import numpy as np
import sys
from PyQt5.QtWidgets import QApplication, QLabel, QWidget
from PyQt5.QtGui import QImage, QPixmap, QPainter, QColor, QPen
from PyQt5.QtCore import QTimer, Qt, QPoint
import time

mp_face_mesh = mp.solutions.face_mesh


class CameraFeed(QWidget):
    def __init__(self):
        super().__init__()

        self.setWindowTitle("Burun İmleci")
        self.setGeometry(100, 100, 1280, 720)

        # Kamera görüntüsü için QLabel
        self.camera_label = QLabel(self)
        self.camera_label.setFixedSize(320, 240)
        self.camera_label.setStyleSheet("border: 1px solid black;")
        self.camera_label.move((self.width() - self.camera_label.width()) // 2, 20)

        # Arayüz alanı
        self.interface_label = QLabel(self)
        self.interface_label.setFixedSize(1280, 400)
        self.interface_label.setStyleSheet("background-color: lightgray; border: 1px solid black;")
        self.interface_label.move(0, self.camera_label.height() + 40)

        # İmleç pozisyonu bilgisi
        self.cursor_label = QLabel(self)
        self.cursor_label.setText("Burun İmleci Koordinatları: X: 640, Y: 200")
        self.cursor_label.setStyleSheet("font-size: 16px; color: green;")
        self.cursor_label.setGeometry(10, self.interface_label.y() - 30, 400, 30)

        # Tebrikler mesajı
        self.congratulations_label = QLabel(self)
        self.congratulations_label.setText("")
        self.congratulations_label.setStyleSheet("font-size: 24px; color: blue; font-weight: bold;")
        self.congratulations_label.setGeometry(400, self.interface_label.y() + 50, 500, 50)
        self.congratulations_label.setAlignment(Qt.AlignCenter)

        # Buton
        self.button_x, self.button_y = 500, 150  # Butonun başlangıç koordinatları
        self.button_width, self.button_height = 200, 100

        # Timer
        self.timer = QTimer(self)
        self.timer.timeout.connect(self.update_frame)

        # Mediapipe FaceMesh
        self.face_mesh = mp_face_mesh.FaceMesh(refine_landmarks=True, min_detection_confidence=0.7, min_tracking_confidence=0.7)

        # Kamera
        self.cap = cv2.VideoCapture(0)
        self.cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
        self.cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
        self.timer.start(20)

        # Hassasiyet çarpanları
        self.x_sensitivity = 3
        self.y_sensitivity = 5

        # Burun imleci pozisyonu
        self.cursor_x = self.interface_label.width() // 2
        self.cursor_y = self.interface_label.height() // 2

        # Kamera yüzü ortalamak için kesme oranları
        self.crop_x = 0.25
        self.crop_y = 0.15

        # Tıklama işlemi
        self.hover_start_time = None
        self.hover_duration = 3  # 3 saniye

    def update_frame(self):
        ret, frame = self.cap.read()
        if not ret:
            return

        # Görüntüyü yatay eksende çevir
        frame = cv2.flip(frame, 1)

        # Görüntüyü kırp
        h, w, c = frame.shape
        x_start = int(w * self.crop_x)
        x_end = w - x_start
        y_start = int(h * self.crop_y)
        y_end = h - y_start
        cropped_frame = frame[y_start:y_end, x_start:x_end]

        # BGR -> RGB
        rgb_frame = cv2.cvtColor(cropped_frame, cv2.COLOR_BGR2RGB)

        # Mediapipe ile yüz analizi
        results = self.face_mesh.process(rgb_frame)

        if results.multi_face_landmarks:
            for face_landmarks in results.multi_face_landmarks:
                h, w, c = cropped_frame.shape
                nose_tip = face_landmarks.landmark[1]

                new_cursor_x = int(nose_tip.x * self.interface_label.width())
                new_cursor_y = int(nose_tip.y * 400)

                if new_cursor_x != self.cursor_x or new_cursor_y != self.cursor_y:
                    self.cursor_x = int((new_cursor_x - self.interface_label.width() // 2) * self.x_sensitivity + self.interface_label.width() // 2)
                    self.cursor_y = int((new_cursor_y - self.interface_label.height() // 2) * self.y_sensitivity + self.interface_label.height() // 2)

                # Sınırlandırma
                self.cursor_x = max(0, min(self.cursor_x, self.interface_label.width()))
                self.cursor_y = max(0, min(self.cursor_y, 400))

                # Tıklama işlemi kontrolü
                self.check_hover()

                # İmleç pozisyonu bilgisini güncelle
                self.cursor_label.setText(f"Burun İmleci Koordinatları: X: {self.cursor_x}, Y: {self.cursor_y}")
                self.repaint_interface_label(self.cursor_x, self.cursor_y)

        # Kamera görüntüsünü QLabel'e aktar
        h, w, ch = rgb_frame.shape
        bytes_per_line = ch * w
        qt_image = QImage(rgb_frame.data, w, h, bytes_per_line, QImage.Format_RGB888)
        pixmap = QPixmap.fromImage(qt_image)
        self.camera_label.setPixmap(pixmap)

    def check_hover(self):
        # İmleç buton üzerinde mi?
        if self.button_x <= self.cursor_x <= self.button_x + self.button_width and \
                self.button_y <= self.cursor_y <= self.button_y + self.button_height:
            if self.hover_start_time is None:
                self.hover_start_time = time.time()
            elif time.time() - self.hover_start_time >= self.hover_duration:
                self.show_congratulations()
                self.hover_start_time = None  # Tekrar tıklamayı engellemek için sıfırla
        else:
            self.hover_start_time = None

    def show_congratulations(self):
        self.congratulations_label.setText("Tebrikler!")  # Tebrikler mesajını göster
        QTimer.singleShot(2000, lambda: self.congratulations_label.setText(""))  # Mesajı 2 saniye sonra sil

    def repaint_interface_label(self, x, y):
        pixmap = QPixmap(self.interface_label.size())
        pixmap.fill(Qt.transparent)

        painter = QPainter(pixmap)

        # Buton çiz
        painter.setBrush(QColor(0, 255, 0))
        painter.drawRect(self.button_x, self.button_y, self.button_width, self.button_height)

        # İmleç çemberi çiz
        painter.setBrush(QColor(255, 0, 0))
        painter.setPen(Qt.NoPen)
        painter.drawEllipse(QPoint(x, y), 20, 20)

        # Animasyon - Çevresel sınır dolması
        if self.hover_start_time:
            elapsed = time.time() - self.hover_start_time
            progress = int((elapsed / self.hover_duration) * 360)
            pen = QPen(QColor(0, 0, 255), 3)
            painter.setPen(pen)
            painter.drawArc(x - 30, y - 30, 60, 60, 0, progress * 16)

        painter.end()
        self.interface_label.setPixmap(pixmap)

    def keyPressEvent(self, event):
        if event.key() == Qt.Key_Q:
            self.close()

    def closeEvent(self, event):
        self.cap.release()
        self.face_mesh.close()
        event.accept()


if __name__ == "__main__":
    app = QApplication([])
    main_window = CameraFeed()
    main_window.show()
    sys.exit(app.exec_())

I0000 00:00:1733697385.551510 2844555 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 89.3), renderer: Apple M2
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1733697385.553243 2844693 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1733697385.558110 2844693 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1733697387.251958 2844695 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.
2024-12-09 01:36:27.292 Python[65435:2844555] +[IMKClient subclass]: chose IMKClient_Modern
2024-12-09 01:36:27.292 Python[65435:2844555] +[IMKInputSession subclass]: chose IMKInputSession_Modern


SystemExit: 0

/Users/ulasyildiz/Documents/Gesture-Based Interaction/.venv/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
